In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from src.data_utils.dataset import build_datset_train_val
from src.rnn.gru_prototipes import build_gru_model
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf

In [2]:
train_dataset, val_dataset = build_datset_train_val(split=0.8, batch_size=256)

train split: 28160 | val split: 6656


In [3]:
def objective(trial):
    tf.keras.backend.clear_session()
    model = build_gru_model(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=5, callbacks=get_predefine_callbacks(model_name="v1"))
    print(model.summary())

    val_loss = model.evaluate(val_dataset)[0]
    return  val_loss

In [4]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1, gc_after_trial=True, show_progress_bar=True)

[I 2023-08-25 13:02:35,685] A new study created in memory with name: no-name-97a1b194-0d4d-479e-8a86-9d8b21f1dbb5


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/20
161/161 [==============================] - 237s 1s/step - loss: 4.0374 - accuracy: 0.6000 - sparse_levenshtein_v1: 0.4058 - val_loss: 3.8640 - val_accuracy: 0.7161 - val_sparse_levenshtein_v1: 0.2933
Epoch 2/20
161/161 [==============================] - 189s 1s/step - loss: 2.2757 - accuracy: 0.7181 - sparse_levenshtein_v1: 0.2909 - val_loss: 1.2955 - val_accuracy: 0.7180 - val_sparse_levenshtein_v1: 0.2921
Epoch 3/20
161/161 [==============================] - 181s 1s/step - loss: 1.2641 - accuracy: 0.7197 - sparse_levenshtein_v1: 0.2905 - val_loss: 1.2476 - val_accuracy: 0.7183 - val_sparse_levenshtein_v1: 0.2937
Epoch 4/20
155/161 [===========================>..] - ETA: 7s - loss: 1.2263 - accuracy: 0.7203 - sparse_levenshtein_v1: 0.2915[W 2023-08-25 13:16:03,534] Trial 0 failed with parameters: {'recurrent_dropout_1': 0.22246794762179156, 'encoder_rnn_layers': 1, 'recurrent_dropout_2': 0.1349530195883304, 'recurrent_dropout_3': 0.498976149121026, 'dropout': 0.029861675869

KeyboardInterrupt: 

# Train best model so far

In [ ]:
tf.keras.backend.clear_session()

trials = study.best_trials

for index, trial in enumerate(trials):
    print(f"Best model: {index+1}")
    model_name = "v1"

    model = build_gru_model(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=5000, callbacks=get_predefine_callbacks(model_name=model_name))
    print(model.summary())

    print('validation loss: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))

    model.load_weights(f"../best_model/prototype/{model_name}")

    print(f"Score in Validation: {model.evaluate(val_dataset)}")

In [ ]:
from src.export_tf_lite.models import TFLiteModel

tf_lite_base_model = TFLiteModel(model)

In [ ]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflitemodel_base)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.allow_custom_ops = True
keras_model_converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = keras_model_converter.convert()

# Save model to disk with the selected columns

In [ ]:
import json

with open("../models/model.tflite", "wb") as f:    
    f.write(tflite_model)
infargs = {"selected_columns" : list(FEATURE_COLUMNS)}

with open("../models/inference_args.json", "w") as json_file:
    json.dump(infargs, json_file)

In [ ]:
!zip submission.zip  '../models/model.tflite' '../models/inference_args.json'

# Test exported model

In [ ]:
# from batch 1

source_batch, target_batch = next(iter(val_dataset))[0]

In [ ]:
REQUIRED_SIGNATURE = "serving_default"
REQUIRED_OUTPUT = "outputs"

interpreter = tf.lite.Interpreter("../models/model.tflite")

with open ("../data/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)

rev_character_map = {j:i for i,j in character_map.items()}
found_signatures = list(interpreter.get_signature_list().keys())

if REQUIRED_SIGNATURE not in found_signatures:
    raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner(REQUIRED_SIGNATURE)

prediction_str = ""
for source_element, target_element in zip(source_batch, target_batch):
    output = prediction_fn(inputs=source_element)

    print("generated: ", "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)]))
    print("target: ", "".join([rev_character_map.get(s, "") for s in target_element.numpy()]))